<h1><center>Temporal Analysis</center></h1>

## (A) Tweets

In [ ]:
#Connecting to the MongoDB database 'EngTweetsDb' where the tweets are stored
import pymongo
MONGO_HOST = 'mongodb://localhost/EngTweetsDb'
conn = pymongo.MongoClient(MONGO_HOST)["EngTweetsDb"]["en_tweets_col"]
conn.count()

In [ ]:
#I want to show the location of the tweet along with its date

#Creating a list of all the locations 
location_id =(list(conn.find({},{"user.location":1})))
print(len(location_id))
location_only = []
for i in range(len(location_id)):
    location_str = location_id[i]["user"]["location"]
    location_only.append(location_str)
    
print(location_only)

In [ ]:
##Extracting the 'timestamp_ms' information from the stored tweets
tweet_timestamp= (list(conn.find({},{"timestamp_ms":1})))
tweet_timestamp
#Timestamp is now being converted from milliseconds (ms) to seconds (s)
timestamp_s_only = []
for i in range(len(tweet_timestamp)):
    time_s = tweet_timestamp[i]["timestamp_ms"]
    timestamp_s_only.append(int(time_s)/1000)
    
print(timestamp_s_only)

In [ ]:
#Converting timestamp in seconds to actual date-time format
from datetime import datetime
Date_Time_List = []
for i in range(len(timestamp_s_only)):
    temp = datetime.fromtimestamp(timestamp_s_only[i])
    Date_Time_List.append(temp)

In [ ]:
#Creating a DataFrame of Date and Location
import pandas as pd
DF = pd.DataFrame({'Date' : Date_Time_List,
                    'Location': location_only})
DF

In [ ]:
#Converting Date_Time_List to a dictionary format to access the date as key and location as value
dd = {st:Date_Time_List.count(st) for st in Date_Time_List}
type(dd)

In [ ]:
#Accesing date as dictionary keys and location as dictionary values
a1, b1 = list(dd.keys()), list(dd.values())

In [ ]:
#Using 'plotly' package to plot the date, time and location values for tweets
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np

l = []
for i in range(len(list(DF['Location']))):
    data = [go.Scatter(x=a1, y=b1, marker = dict(size='16',
        color = np.random.randn(10000), #set color equal to a variable
        colorscale='Viridis',
        showscale=True), mode = 'markers', text=DF['Location'])]
    l.append(data);
    
layout = dict(
    title='Temporal Analysis of Tweets',
    hovermode= 'closest',
    xaxis=dict(
        title='Time',
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1d',
                     step='day',
                     stepmode='backward'),
                dict(count=2,
                     label='2d',
                     step='day',
                     stepmode='backward'),
                dict(count=3,
                     label='3d',
                     step='day',
                     stepmode='backward'),
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(),
        type='date'
    ),
    yaxis=dict(title='Frequency of Tweets')
)
fig = dict(data=data,layout = layout)
py.iplot(fig)

# (B) News Article

In [ ]:
#Connecting to the MongoDB database 'News' where the news articles are stored
MONGO_HOST1 = 'mongodb://localhost/News'
conn1 = pymongo.MongoClient(MONGO_HOST1)["News"]["articles"]
conn1.count()

In [ ]:
#Passing the list of five most frequently occurring entities in tweets
top_five = ['hindu','justin','bieber','#bbmas','trump']

In [ ]:
#Extracting the 'content' information from the stored news articles
content_id =(list(conn1.find({})))
print(content_id[3])

In [ ]:
#Functions to extract content and location data from the collected news articles
def timestampprocessor(s):    
    start="'published_at': datetime.datetime("
    end=", tzinfo="
    timestamp=(s.split(start))[1].split(end)[0]
    return eval(timestamp.strip())

def locationprocessor(s):
    start="'locations':"
    end="'logo_url':"
    location=(s.split(start))[1].split(end)[0]
    return eval(location.strip())

In [ ]:
#Creating a date-time list 
date_list = []
for i in range(len(content_id)):
    s=content_id[i][list(content_id[i].keys())[1]]
    date=timestampprocessor(s)
    print(str(date1[0:6]).replace(")","").replace("(",""))
    d = str(date[0:3]).replace(")","").replace("(","")
    x= '-'.join(d.split(", "))
    d= str(date[3:6]).replace(")","").replace("(","")
    y= ':'.join(d.split(", "))
    date_list.append(x+" "+y)
print(date_list)

In [ ]:
#For creating a location list, only the country data has been extracted and used
country=[]
for i in range(len(content_id)):
    s=content_id[i][list(content_id[i].keys())[1]]
    location=locationprocessor(s)
    if len(location[0]) == 0:
        country.append(None)
        i = i + 1
        print('Record skipped.')
    else:
        country.append(location[0][0]['country'])
print(country)

In [ ]:
#Storing date_list as date
date = date_list

In [ ]:
#Creating a DataFrame of Date and Location
import pandas as pd
DF = pd.DataFrame({'Date' : date,
                    'Location': country})

In [ ]:
#Deriving the values of Date & Location column of DataFrame
a1 = list(DF['Date'])
b1 = list(DF['Location'])

In [ ]:
#Using 'Plotly' package to plot the date, time and location values for news articles
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np

l = []
for i in range(len(list(DF['Location']))):
    data = [go.Scatter(x=a1, marker = dict(size='16',
        color ="" ), mode = 'markers', text=DF['Location'])]
    l.append(data);
    
layout = dict(
    title='Temporal Analysis of News Articles',
    hovermode= 'closest',
    xaxis=dict(
        title='Time',
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1d',
                     step='day',
                     stepmode='backward'),
                dict(count=2,
                     label='2d',
                     step='day',
                     stepmode='backward'),
                dict(count=3,
                     label='3d',
                     step='day',
                     stepmode='backward'),
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(),
        type='date'
    )
)
fig = dict(data=data,layout = layout)
py.iplot(fig)